In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import dual_annealing as anneal_algo
from mpl_toolkits.mplot3d import Axes3D
import scipy.integrate as integrate 
import pysindy as ps

# Atmospheric attractor parameters
r = 3
sigma = 1.1
beta = -5.0
eps = 0.01
k1 = 5
k2 = 1
mu1 = eps*k1
mu2 = -eps*k2
alpha = -2.0
omega = 3.0

dt = 0.01
T = 500
t = np.arange(0,T+dt,dt)
tmax = len(t)-1

def oscillator_deriv(x_y_z, t0):
    x, y, z = x_y_z
    return [mu1*x + sigma*x*y, mu2*y + (omega + alpha*y + beta*z)*z - sigma*x**2, mu2*z - (omega + alpha*y + beta*z)*y]

x0_train = [0.1, -0.1, 0.1]
x_t = integrate.odeint(oscillator_deriv, x0_train, t,rtol=10**(-12),atol=10**(-12)*np.ones_like(x0_train))

# Add some noise to the data
mean = 0.0
variance = 0.0
np.random.seed(502)
noise = np.random.normal(mean,variance,x_t.shape[0])
for i in range(r):
    x_t[:,i] = x_t[:,i] + noise

In [2]:
# Define all the complicated matrices in the optimization
poly_order = 2
Nr = int((r**2 + 3*r)/2.0)
mean = 0.0
q = 0
constraint_zeros = np.zeros((r + r*(r-1) + int(r*(r-1)*(r-2)/6.0)))
constraint_matrix = np.zeros((r + r*(r-1) + int(r*(r-1)*(r-2)/6.0), int(r*(r**2+3*r)/2)))

# Set coefficients adorning terms like a_i^3 to zero
for i in range(r):
    constraint_matrix[q, r*(int((r**2+3*r)/2.0)-r) + i*(r+1)] = 1.0
    q = q + 1

# Set coefficients adorning terms like a_ia_j^2 to be antisymmetric
for i in range(r):
    for j in range(i+1, r):
        constraint_matrix[q, r*(int((r**2+3*r)/2.0)-r+j)+i] = 1.0
        constraint_matrix[q, r*(r+j-1)+j+r*int(i*(2*r-i-3)/2.0)] = 1.0
        q = q + 1
for i in range(r):
     for j in range(0, i):
        constraint_matrix[q, r*(int((r**2+3*r)/2.0)-r+j)+i] = 1.0
        constraint_matrix[q, r*(r+i-1)+j+r*int(j*(2*r-j-3)/2.0)] = 1.0
        q = q + 1

# Set coefficients adorning terms like a_ia_ja_k to be antisymmetric
for i in range(r):
    for j in range(i+1, r):
        for k in range(j+1, r):
            constraint_matrix[q, r*(r+k-1)+i+r*int(j*(2*r-j-3)/2.0)] = 1.0
            constraint_matrix[q, r*(r+k-1)+j+r*int(i*(2*r-i-3)/2.0)] = 1.0
            constraint_matrix[q, r*(r+j-1)+k+r*int(i*(2*r-i-3)/2.0)] = 1.0
            q = q + 1
# delta_{il}delta_{jk}
PL_tensor = np.zeros((r, r, r, Nr))
for i in range(r):
    for j in range(r):
        for k in range(r):
            for l in range(Nr):
                if i == l and j == k:
                    PL_tensor[i, j, k, l] = 1.0

# Now symmetrize PL
for i in range(r):
    for j in range(Nr):
        PL_tensor[:, :, i, j] = 0.5 * (PL_tensor[:, :, i, j] + PL_tensor[:, :, i, j].T)

PQ_tensor = np.zeros((r, r, Nr))
for i in range(r):
    # Off diagonal terms
    for j in range(i+1, r):
        PQ_tensor[i, j, int((i+1)/2.0*(2*r-i)) + j - 1 - i] = 1.0  # opt_m[k]  

    # diagonal terms
    PQ_tensor[i, i, Nr - r + i] = 1.0  # opt_m[j]

# Now symmetrize PQ
for j in range(Nr):
    PQ_tensor[:, :, j] = 0.5 * (PQ_tensor[:, :, j] + PQ_tensor[:, :, j].T)

x, y, z = x_t.T

if poly_order == 2:
    library_functions = [lambda x:x, lambda x, y:x*y, lambda x:x**2]
    library_function_names = [lambda x:x, lambda x, y:x+y, lambda x:x+x]
else:
    library_functions = [lambda x:x]
    library_function_names = [lambda x:x]
sindy_library = ps.CustomLibrary(library_functions=library_functions,
                                    function_names=library_function_names)

In [ ]:
# Define the hyperparams and perform the optimization (regular prox-gradient)
eta = 1.0e20
gamma = 1.0e-3 * eta
beta = 0.9 * eta
threshold = 0.0
sindy_opt = ps.clSR3(threshold=threshold, eta=eta, alpha_m=gamma, alpha_A=beta, vtol=1e-3, accel=True,
                            PL=PL_tensor, PQ=PQ_tensor, max_iter=400, tol=1e-10,
                            thresholder="l1", eigmin=-1e50, eigmax=-1e9,
                            constraint_lhs=constraint_matrix,
                            constraint_rhs=constraint_zeros,
                            constraint_order="feature")

model = ps.SINDy(
            optimizer=sindy_opt,
            feature_library=sindy_library,
            differentiation_method=ps.FiniteDifference(drop_endpoints=True),
        )
model.fit(x_t, t=t)
Xi = model.coefficients().T

# Use optimal m, and calculate eigenvalues(PW) to see if identified model is stable
opt_m = sindy_opt.m_history_[-1]
mPQ = np.zeros(PL_tensor.shape)
for i in range(r):
    for j in range(i+1, r):
        mPQ[i, j, :, int((i+1)/2.0*(2*r-i)) + j - 1 - i] = opt_m
for i in range(r):
    mPQ[i, i, :, Nr - r + i] = opt_m
for i in range(r):
    for j in range(Nr):
        mPQ[:, :, i, j] = 0.5 * (mPQ[:, :, i, j] + mPQ[:, :, i, j].T)     
P_tensor = PL_tensor - mPQ
As = np.tensordot(P_tensor, Xi, axes=([3, 2], [0, 1]))
eigvals, eigvecs = np.linalg.eig(As)
print('proxgrad, As eigvals: ', eigvals)
print(opt_m)
smallest_eigval = np.sort(eigvals)[0]
largest_eigval = np.sort(eigvals)[-1]
L = np.tensordot(PL_tensor, Xi, axes=([3,2],[0,1]))
Q = np.tensordot(PQ_tensor, Xi, axes=([2],[0]))
d = np.dot(L, opt_m) + np.dot(np.tensordot(Q, opt_m, axes=([2],[0])), opt_m)
Rm = np.sqrt(smallest_eigval/largest_eigval)*np.linalg.norm(d)
print(d, Rm)

model.print()
print(np.dot(constraint_matrix, (sindy_opt.history_[0]).T.flatten()))
print(np.dot(constraint_matrix, (sindy_opt.history_[1]).T.flatten()))

0.0009933733874922337 0.0 0.0
0.0009933733874922337 0.0049899177957437035 0.0
0.0009933733874922337 0.004989634828696391 0.0
0.0009933733874922337 0.004989344631728485 0.0
0.0009933733874922337 0.004989051400776776 0.0
0.0009933733874922337 0.004988756495532127 0.0
0.0009933733874922337 0.004988460530660232 0.0
0.0009933733874922337 0.0049881638368862715 0.0
0.0009933733874922337 0.0049878666127594505 0.0
0.0009933733874922337 0.004987568986872988 0.0
0.0009933733874922337 0.004987271047301544 0.0
0.0009933733874922337 0.004986972857017397 0.0
0.0009933733874922337 0.004986674462607402 0.0
0.0009933733874922337 0.004986375899506796 0.0
0.0009933733874922337 0.0049860771952968475 0.0
0.0009933733874922337 0.004985778371867178 0.0
0.0009933733874922337 0.004985479446881438 0.0
0.0009933733874922337 0.004985180434799683 0.0
0.0009933733874922337 0.004984881347609114 0.0
0.0009933733874922337 0.0049845821953576315 0.0
0.0009933733874922337 0.004984282986550824 0.0
0.0009933733874922337 0.0

0.0009933733874922337 0.004937967844606761 0.0
0.0009933733874922337 0.004937670217045243 0.0
0.0009933733874922337 0.004937372606558722 0.0
0.0009933733874922337 0.004937075013156091 0.0
0.0009933733874922337 0.004936777436845883 0.0
0.0009933733874922337 0.004936479877636607 0.0
0.0009933733874922337 0.00493618233553662 0.0
0.0009933733874922337 0.004935884810554045 0.0
0.0009933733874922337 0.004935587302696931 0.0
0.0009933733874922337 0.004935289811973111 0.0
0.0009933733874922337 0.004934992338390306 0.0
0.0009933733874922337 0.004934694881956142 0.0
0.0009933733874922337 0.004934397442678005 0.0
0.0009933733874922337 0.004934100020563182 0.0
0.0009933733874922337 0.004933802615618919 0.0
0.0009933733874922337 0.00493350522785215 0.0
0.0009933733874922337 0.004933207857269923 0.0
0.0009933733874922337 0.004932910503878916 0.0
0.0009933733874922337 0.00493261316768581 0.0
0.0009933733874922337 0.004932315848697156 0.0
0.0009933733874922337 0.00493201854691941 0.0
0.000993373387492

0.0009933733874922337 0.004886147143214303 0.0
0.0009933733874922337 0.00488585256370187 0.0
0.0009933733874922337 0.004885558001734216 0.0
0.0009933733874922337 0.004885263457311515 0.0
0.0009933733874922337 0.0048849689304340145 0.0
0.0009933733874922337 0.004884674421101855 0.0
0.0009933733874922337 0.004884379929315165 0.0
0.0009933733874922337 0.00488408545507407 0.0
0.0009933733874922337 0.004883790998378805 0.0
0.0009933733874922337 0.004883496559229444 0.0
0.0009933733874922337 0.004883202137626048 0.0
0.0009933733874922337 0.004882907733568797 0.0
0.0009933733874922337 0.004882613347057776 0.0
0.0009933733874922337 0.004882318978093003 0.0
0.0009933733874922337 0.004882024626674672 0.0
0.0009933733874922337 0.004881730292802826 0.0
0.0009933733874922337 0.004881435976477316 0.0
0.0009933733874922337 0.004881141677698539 0.0
0.0009933733874922337 0.004880847396466301 0.0
0.0009933733874922337 0.004880553132780749 0.0
0.0009933733874922337 0.0048802588866418035 0.0
0.00099337338

0.0009933733874922337 0.004834862724460168 0.0
0.0009933733874922337 0.004834571211911429 0.0
0.0009933733874922337 0.00483427971684432 0.0
0.0009933733874922337 0.004833988239258308 0.0
0.0009933733874922337 0.004833696779152524 0.0
0.0009933733874922337 0.004833405336526332 0.0
0.0009933733874922337 0.0048331139113791145 0.0
0.0009933733874922337 0.004832822503710124 0.0
0.0009933733874922337 0.004832531113518613 0.0
0.0009933733874922337 0.004832239740803962 0.0
0.0009933733874922337 0.0048319483855654806 0.0
0.0009933733874922337 0.00483165704780242 0.0
0.0009933733874922337 0.004831365727514166 0.0
0.0009933733874922337 0.004831074424699833 0.0
0.0009933733874922337 0.004830783139358935 0.0
0.0009933733874922337 0.00483049187149058 0.0
0.0009933733874922337 0.004830200621094225 0.0
0.0009933733874922337 0.004829909388169037 0.0
0.0009933733874922337 0.0048296181727143115 0.0
0.0009933733874922337 0.004829326974729392 0.0
0.0009933733874922337 0.004829035794213628 0.0
0.00099337338

0.0009933733874922337 0.004784113525227359 0.0
0.0009933733874922337 0.004783825060210939 0.0
0.0009933733874922337 0.004783536612535145 0.0
0.0009933733874922337 0.004783248182198987 0.0
0.0009933733874922337 0.004782959769201825 0.0
0.0009933733874922337 0.004782671373542414 0.0
0.0009933733874922337 0.0047823829952202655 0.0
0.0009933733874922337 0.00478209463423415 0.0
0.0009933733874922337 0.004781806290583445 0.0
0.0009933733874922337 0.004781517964267118 0.0
0.0009933733874922337 0.0047812296552841365 0.0
0.0009933733874922337 0.004780941363633828 0.0
0.0009933733874922337 0.004780653089315246 0.0
0.0009933733874922337 0.004780364832327524 0.0
0.0009933733874922337 0.0047800765926696645 0.0
0.0009933733874922337 0.004779788370340788 0.0
0.0009933733874922337 0.004779500165340071 0.0
0.0009933733874922337 0.004779211977666633 0.0
0.0009933733874922337 0.004778923807319391 0.0
0.0009933733874922337 0.0047786356542977295 0.0
0.0009933733874922337 0.004778347518600458 0.0
0.00099337

...

0.0009933733874922337 0.004444255181698083 0.0
0.0009933733874922337 0.004443987187060334 0.0
0.0009933733874922337 0.0044437192085765975 0.0
0.0009933733874922337 0.004443451246245976 0.0
0.0009933733874922337 0.0044431833000673 0.0
0.0009933733874922337 0.004442915370039425 0.0
0.0009933733874922337 0.0044426474561618315 0.0
0.0009933733874922337 0.004442379558433472 0.0
0.0009933733874922337 0.004442111676853014 0.0
0.0009933733874922337 0.004441843811419914 0.0
0.0009933733874922337 0.004441575962132969 0.0
0.0009933733874922337 0.0044413081289909175 0.0
0.0009933733874922337 0.004441040311993754 0.0
0.0009933733874922337 0.004440772511139613 0.0
0.0009933733874922337 0.004440504726427645 0.0
0.0009933733874922337 0.004440236957857177 0.0
0.0009933733874922337 0.004439969205427033 0.0
0.0009933733874922337 0.004439701469136432 0.0
0.0009933733874922337 0.004439433748984348 0.0
0.0009933733874922337 0.004439166044969649 0.0
0.0009933733874922337 0.004438898357091617 0.0
0.0009933733

0.0009933733874922337 0.0043973360344184784 0.0
0.0009933733874922337 0.0043970708680286285 0.0
0.0009933733874922337 0.004396805717623273 0.0
0.0009933733874922337 0.004396540583201597 0.0
0.0009933733874922337 0.004396275464762676 0.0
0.0009933733874922337 0.00439601036230518 0.0
0.0009933733874922337 0.00439574527582869 0.0
0.0009933733874922337 0.004395480205331718 0.0
0.0009933733874922337 0.004395215150813647 0.0
0.0009933733874922337 0.00439495011227319 0.0
0.0009933733874922337 0.004394685089709934 0.0
0.0009933733874922337 0.00439442008312259 0.0
0.0009933733874922337 0.004394155092510057 0.0
0.0009933733874922337 0.0043938901178715625 0.0
0.0009933733874922337 0.0043936251592060355 0.0
0.0009933733874922337 0.004393360216512541 0.0
0.0009933733874922337 0.004393095289790481 0.0
0.0009933733874922337 0.004392830379038175 0.0
0.0009933733874922337 0.004392565484255466 0.0
0.0009933733874922337 0.004392300605440743 0.0
0.0009933733874922337 0.004392035742593369 0.0
0.00099337338

0.0009933733874922337 0.0043509120579458445 0.0
0.0009933733874922337 0.004350649690126187 0.0
0.0009933733874922337 0.004350387338123316 0.0
0.0009933733874922337 0.00435012500193597 0.0
0.0009933733874922337 0.004349862681563749 0.0
0.0009933733874922337 0.004349600377005216 0.0
0.0009933733874922337 0.004349338088259497 0.0
0.0009933733874922337 0.004349075815325892 0.0
0.0009933733874922337 0.004348813558203536 0.0
0.0009933733874922337 0.004348551316890661 0.0
0.0009933733874922337 0.00434828909138735 0.0
0.0009933733874922337 0.00434802688169177 0.0
0.0009933733874922337 0.0043477646878037025 0.0
0.0009933733874922337 0.0043475025097218034 0.0
0.0009933733874922337 0.004347240347445252 0.0
0.0009933733874922337 0.0043469782009729705 0.0
0.0009933733874922337 0.004346716070304193 0.0
0.0009933733874922337 0.004346453955437648 0.0
0.0009933733874922337 0.004346191856372977 0.0
0.0009933733874922337 0.004345929773108647 0.0
0.0009933733874922337 0.004345667705643882 0.0
0.0009933733

0.0009933733874922337 0.004305237667174703 0.0
0.0009933733874922337 0.004304978052868966 0.0
0.0009933733874922337 0.004304718454214995 0.0
0.0009933733874922337 0.0043044588712110605 0.0
0.0009933733874922337 0.004304199303856847 0.0
0.0009933733874922337 0.004303939752150949 0.0
0.0009933733874922337 0.004303680216092934 0.0
0.0009933733874922337 0.004303420695681122 0.0
0.0009933733874922337 0.004303161190915536 0.0
0.0009933733874922337 0.004302901701794749 0.0
0.0009933733874922337 0.004302642228317605 0.0
0.0009933733874922337 0.004302382770483234 0.0
0.0009933733874922337 0.004302123328291126 0.0
0.0009933733874922337 0.0043018639017396625 0.0
0.0009933733874922337 0.004301604490828597 0.0
0.0009933733874922337 0.004301345095556535 0.0
0.0009933733874922337 0.0043010857159228606 0.0
0.0009933733874922337 0.0043008263519261295 0.0
0.0009933733874922337 0.004300567003565958 0.0
0.0009933733874922337 0.004300307670841244 0.0
0.0009933733874922337 0.004300048353750862 0.0
0.0009933

0.0009933733874922337 0.004260042632295548 0.0
0.0009933733874922337 0.004259785742711601 0.0
0.0009933733874922337 0.004259528868615826 0.0
0.0009933733874922337 0.0042592720100071 0.0
0.0009933733874922337 0.0042590151668831875 0.0
0.0009933733874922337 0.004258758339243973 0.0
0.0009933733874922337 0.004258501527089379 0.0
0.0009933733874922337 0.004258244730418042 0.0
0.0009933733874922337 0.004257987949228156 0.0
0.0009933733874922337 0.004257731183519575 0.0
0.0009933733874922337 0.0042574744332905725 0.0
0.0009933733874922337 0.0042572176985415525 0.0
0.0009933733874922337 0.004256960979271054 0.0
0.0009933733874922337 0.004256704275477527 0.0
0.0009933733874922337 0.004256447587160001 0.0
0.0009933733874922337 0.004256190914318666 0.0
0.0009933733874922337 0.004255934256952 0.0
0.0009933733874922337 0.004255677615059042 0.0
0.0009933733874922337 0.004255420988638053 0.0
0.0009933733874922337 0.004255164377689543 0.0
0.0009933733874922337 0.00425490778221153 0.0
0.00099337338749

In [ ]:
2*np.sqrt(6)*10*11 - Rm

In [ ]:
plt.figure(200)
plt.semilogy(sindy_opt.objective_history[2:])

In [ ]:
# Plot the results
fig,ax = plt.subplots(1,1,subplot_kw={'projection': '3d'},figsize=(16,12))
ax.plot(x_t[:,0], x_t[:,1], x_t[:,2],linewidth=2)
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-20
integrator_keywords['h0'] = 1e-5
x_pred = model.simulate(x0_train, t, integrator_kws=integrator_keywords)
if np.all(eigvals < 0.0):
    ax.plot(x_pred[:tmax,0], x_pred[:tmax,1], x_pred[:tmax,2], 'k--', linewidth=1)
    #ax.set_title('m = [{0:.1f}, '.format(opt_m[0]) + '{0:.1f}, '.format(opt_m[1]) + '{0:0.1f}]'.format(opt_m[2]), fontsize=20)
else:
    ax.plot(x_pred[:tmax,0], x_pred[:tmax,1], x_pred[:tmax,2], 'g--', linewidth=2)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])
plt.savefig('atmospheric_3d.pdf')

In [ ]:
# Check energy-preserving constraint is satisfied (Qijk + Qjik + Qkij = 0)
# and Cw = d

Ls = np.tensordot(PL_tensor, Xi, axes=([3,2],[0,1]))
Q = np.tensordot(PQ_tensor, Xi, axes=([2],[0]))
Qsum = np.sum(Q + np.transpose(Q, [1, 0, 2]) + np.transpose(Q, [2, 1, 0]))
print(Q + np.transpose(Q, [0, 2, 1]) + np.transpose(Q, [2, 1, 0]))
print(np.dot(constraint_matrix, Xi.flatten()))

In [ ]:
# Repeat with accelerated prox-gradient descent
# Define the hyperparams and perform the optimization (regular prox-gradient)
sindy_opt = ps.clSR3(threshold=threshold, eta=eta, alpha_m=gamma, alpha_A=beta, vtol=1e-5, accel=True,
                            PL=PL_tensor, PQ=PQ_tensor, max_iter=40000, tol=1e-10,
                            thresholder="l1", eigmin=-1e50, eigmax=-1e-5,
                            constraint_lhs=constraint_matrix,
                            constraint_rhs=constraint_zeros,
                            constraint_order="feature")

model = ps.SINDy(
            optimizer=sindy_opt,
            feature_library=sindy_library,
            differentiation_method=ps.FiniteDifference(drop_endpoints=True),
        )
model.fit(x_t, t=t)
Xi = model.coefficients().T

# Use optimal m, and calculate eigenvalues(PW) to see if identified model is stable
opt_m = sindy_opt.m_history_[-1]
mPQ = np.zeros(PL_tensor.shape)
for i in range(r):
    for j in range(i+1, r):
        mPQ[i, j, :, int((i+1)/2.0*(2*r-i)) + j - 1 - i] = opt_m
for i in range(r):
    mPQ[i, i, :, Nr - r + i] = opt_m
for i in range(r):
    for j in range(Nr):
        mPQ[:, :, i, j] = 0.5 * (mPQ[:, :, i, j] + mPQ[:, :, i, j].T)     
P_tensor = PL_tensor - mPQ
As = np.tensordot(P_tensor, Xi, axes=([3, 2], [0, 1]))
eigvals, eigvecs = np.linalg.eig(As)
print('proxgrad, As eigvals: ', eigvals)
print(opt_m)
smallest_eigval = np.sort(eigvals)[0]
largest_eigval = np.sort(eigvals)[-1]
L = np.tensordot(PL_tensor, Xi, axes=([3,2],[0,1]))
Q = np.tensordot(PQ_tensor, Xi, axes=([2],[0]))
d = np.dot(L, opt_m) + np.dot(np.tensordot(Q, opt_m, axes=([2],[0])), opt_m)
Rm = np.sqrt(smallest_eigval/largest_eigval)*np.linalg.norm(d)
print(d, Rm)

model.print()
print(np.dot(constraint_matrix, (sindy_opt.history_[0]).T.flatten()))
print(np.dot(constraint_matrix, (sindy_opt.history_[1]).T.flatten()))

In [ ]:
# Plot the results
fig,ax = plt.subplots(1,1,subplot_kw={'projection': '3d'},figsize=(16,12))
ax.plot(x_t[:,0], x_t[:,1], x_t[:,2],linewidth=2)
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-20
integrator_keywords['h0'] = 1e-5
x_pred = model.simulate(x0_train, t, integrator_kws=integrator_keywords)
if np.all(eigvals < 0.0):
    ax.plot(x_pred[:tmax,0], x_pred[:tmax,1], x_pred[:tmax,2], 'k--', linewidth=1)
    ax.set_title('m = [{0:.1f}, '.format(opt_m[0]) + '{0:.1f}, '.format(opt_m[1]) + '{0:0.1f}]'.format(opt_m[2]), fontsize=20)
else:
    ax.plot(x_pred[:tmax,0], x_pred[:tmax,1], x_pred[:tmax,2], 'g--', linewidth=2)

plt.figure(200)
plt.semilogy(sindy_opt.objective_history[1:])

### Note accelerated prox-gradient only took 5 steps instead of ~4000!!